In [ ]:
! pip install -q litellm tqdm PyMuPDF Pillow instructor openai python-dotenv weaviate-client pandas

In [1]:
import os 
from dotenv import load_dotenv

load_dotenv()

True

In [33]:
def get_gpqa_rag_agent_instruction(MAX_SEARCH_LIMIT, MAX_URL_FETCH):
    return (
        "You are a reasoning assistant with the ability to perform web searches and retrieve webpage content to help "
        "you answer the user’s question accurately. You have special tools:\n\n"
        "- To perform a search: write <|begin_search_query|> your query here <|end_search_query|>.\n"
        "Then, the system will call the web search API with your query and return the search results to you in the format <|begin_search_result|> ...search results... <|end_search_result|>.\n"
        "  The search results will contain a list of webpages with titles, URLs, and snippets (but not full content).\n\n"
        "- After receiving the search results, if you need more detailed information from one or more specific URLs, write <|begin_url|> url1, url2, ... <|end_url|>.\n"
        "  The system will fetch the full page content of those URLs and return it to you as <|begin_full_page|> ...full page content... <|end_full_page|>.\n\n"
        f"You can repeat the search process multiple times if necessary. The maximum number of search attempts is limited to {MAX_SEARCH_LIMIT}.\n"
        f"You can fetch up to {MAX_URL_FETCH} URLs for detailed information.\n\n"
        "Once you have all the information you need, continue your reasoning.\n\n"
        "Example:\n"
        "Question: \"What is the energy range of pp III neutrinos?\"\n"
        "Assistant thinking steps:\n"
        "- I might need to look up details about pp III neutrinos.\n\n"
        "Assistant:\n"
        "<|begin_search_query|>pp III neutrino energy spectrum<|end_search_query|>\n\n"
        "(System returns search results)\n\n"
        "Assistant:\n"
        "<|begin_search_result|> ...search results without full page... <|end_search_result|>\n\n"
        "Assistant thinks: The search results mention some URLs. I want full details from one of them.\n\n"
        "Assistant:\n"
        "<|begin_url|>http://example.com/ppIII_neutrino.html<|end_url|>\n\n" 
        "(System returns full page content)\n\n"
        "Assistant:\n"
        "<|begin_full_page|> ...full page content... <|end_full_page|>\n\n"
        "Now the assistant has enough info and can continue reasoning.\n\n"
        "Remember:\n"
        "- Use <|begin_search_query|> to request a web search and end with <|end_search_query|>.\n"
        "- Use <|begin_url|> to request full page content and end with <|end_url|>.\n"
        "- When done retrieving information, continue your reasoning.\n\n"
    )

In [35]:
def get_task_instruction_openqa(question, model_name=None):
    if model_name == 'qwq':
        user_prompt = (
            'Please answer the following question. '
            'You should provide your final answer in the format \\boxed{YOUR_ANSWER}.\n\n'
            f'Question:\n{question}\n\n'
        )
    else:
        user_prompt = (
            'Please answer the following question. You should think step by step to solve it.\n\n'
            'Provide your final answer in the format \\boxed{YOUR_ANSWER}.\n\n'
            f'Question:\n{question}\n\n'
        )
    return user_prompt

In [36]:
question = """
Is 3M a capital-intensive business based on FY2022 data?
"""

In [39]:

user_prompt = get_task_instruction_openqa(question)
instruction = get_gpqa_rag_agent_instruction(5,10)

prompt = [{"role": "user", "content": instruction + user_prompt}]

In [29]:
prompt

[{'role': 'user',
  'content': 'You are a reasoning assistant with the ability to perform web searches and retrieve webpage content to help you answer the user’s question accurately. You have special tools:\n\n- To perform a search: write <|begin_search_query|> your query here <|end_search_query|>.\nThen, the system will call the web search API with your query and return the search results to you in the format <|begin_search_result|> ...search results... <|end_search_result|>.\n  The search results will contain a list of webpages with titles, URLs, and snippets (but not full content).\n\n- After receiving the search results, if you need more detailed information from one or more specific URLs, write <|begin_url|> url1, url2, ... <|end_url|>.\n  The system will fetch the full page content of those URLs and return it to you as <|begin_full_page|> ...full page content... <|end_full_page|>.\n\nYou can repeat the search process multiple times if necessary. The maximum number of search attem

In [44]:
from litellm import completion
response = completion(
        model="openrouter/google/gemini-2.0-flash-thinking-exp:free",
        messages=prompt,
        temperature=0.2
    )
print(response)

17:06:04 - LiteLLM:WARNING: utils.py:318 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.




Request to litellm:
litellm.completion(model='openrouter/google/gemini-2.0-flash-thinking-exp:free', messages=[{'role': 'user', 'content': 'You are a reasoning assistant with the ability to perform web searches and retrieve webpage content to help you answer the user’s question accurately. You have special tools:\n\n- To perform a search: write <|begin_search_query|> your query here <|end_search_query|>.\nThen, the system will call the web search API with your query and return the search results to you in the format <|begin_search_result|> ...search results... <|end_search_result|>.\n  The search results will contain a list of webpages with titles, URLs, and snippets (but not full content).\n\n- After receiving the search results, if you need more detailed information from one or more specific URLs, write <|begin_url|> url1, url2, ... <|end_url|>.\n  The system will fetch the full page content of those URLs and return it to you as <|begin_full_page|> ...full page content... <|end_ful

In [47]:
print(response.choices[0].message.content)

Here's a breakdown of the thought process to answer the question:

1. **Understand the Question:** The core question is whether 3M is capital-intensive in 2022. This requires defining "capital-intensive" and then finding relevant data for 3M in 2022.

2. **Define Capital-Intensive:**  A capital-intensive business requires significant investment in fixed assets (property, plant, and equipment - PP&E) to generate revenue.  Key indicators often involve comparing capital expenditures to revenue or looking at the proportion of fixed assets on the balance sheet.

3. **Initial Search Strategy:**  The most direct approach is to search for information specifically stating whether 3M is capital-intensive.  If that fails, searching for 3M's financial data for 2022 will be necessary.

4. **First Search Attempt:**
   <|begin_search_query|>is 3M a capital intensive business 2022<|end_search_query|>

5. **Analyze Search Results (Simulated):**  Assume the search results provide some general informatio

In [48]:
# Extract the search query with <|begin_search_query|> and <|end_search_query|>
search_query = response.choices[0].message.content.split("<|begin_search_query|>")[1].split("<|end_search_query|>")[0].strip()
print("Search Query:", search_query)



Search Query: is 3M a capital intensive business 2022
